# Расчет роликов по нескольким ЦА
Пример выгрузки показателей рекламных роликов по нескольким целевым аудиториям

## Описание задачи и условий расчета
- Период: 24.07.2023 - 30.07.2023
- ЦА: Женщины 25-54; Все 18-64; Мужчины 25-45 BC
- Место просмотра: Все места (Дом+Дача)
- Каналы: все каналы проекта TV Index
- Ролики: Товарная категория 4: ИНТЕРНЕТ-СУПЕРМАРКЕТЫ; Статус события: Реальный; Блок распространение - сетевой, орибтальный
- Статистики: Quantity, RtgPer, StandRtgPer

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)

# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)

# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники
Получим идентификаторы, которые будут использоваться для формирования условий расчета

In [ ]:
# Получим id товарной категории 4
cats.get_tv_article(levels='4', name=['ИНТЕРНЕТ-СУПЕРМАРКЕТЫ'])

# Для ИНТЕРНЕТ-СУПЕРМАРКЕТЫ это 4926

In [ ]:
# Обратимся к справочнику статусов событий
cats.get_tv_issue_status()

# В условия отчета передадим идентификатор R (реальный)

In [ ]:
# Обратимся к справочнику типов распространения блоков
cats.get_tv_breaks_distribution()

# В условия отчета передадим идентификаторы N (сетевой) и O (орбитальный)

In [ ]:
# В справочнике демографических атрибутов найдем необходимые нам переменные
cats.get_tv_demo_attribute(names=['пол','возраст','группа дохода по россии'])

# Для формирования условий запроса нам понадобятся значения столбца entityName: sex, age, incomeGroupRussia

## Формирование задания
Зададим условия расчета

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'. Можно указать несколько периодов
date_filter = [('2023-07-24', '2023-07-30')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter=None

# Задаем каналы
company_filter = None

# Фильтр программ
program_filter = None

# Фильтр блоков: Блок распространение - сетевой, орибтальный
break_filter = 'breaksDistributionType IN (N,O)'

# Фильтр роликов: Товарная категория 4 - ИНТЕРНЕТ-СУПЕРМАРКЕТЫ; Статус события - Реальный
ad_filter = 'articleLevel4Id IN (4926) and adIssueStatusId = R'

# Указываем список срезов
slices = [
    'advertiserListName', #Список рекламодателей
    'brandListName', #Список брендов
    'subbrandListName', #Список суббрендов
    'modelListName', #Список продуктов
    'articleList4Name', #Список товарных категорий 4 уровня
    'adName', #Ролик
    'adStandardDuration', #Ролик ожидаемая длительность
    'adDistributionTypeName', #Ролик распространение
    'adTypeName', #Ролик тип
    'adId', #Ролик ID
    'adPositionId', #Ролик позиция в блоке
    'adPositionTypeName', #Ролик позиционирование
    'tvCompanyName', #Телекомпания
    'programName', #Программа
    'researchDate', #Дата, обязательный атрибут!
    'adStartTime', #Ролик время начала
    'adPrimeTimeStatusName', #Прайм/не Прайм
    'adSpotId', #Ролик ID выхода, обязательный атрибут! 
]

# Указываем список статистик для расчета
statistics = ['Quantity', 'RtgPer', 'StandRtgPer']

# Задаем условия сортировки: Ролик ID выхода (по возр.)
sortings = {'adSpotId':'ASC'}

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all
    "standRtg" : {
      "useRealDuration" : True, #расчет по реальной длительности ролика.
      "standardDuration" : 20 #стандартная длительность 20 сек.
    }
}

Сформируем словарь с целевыми аудиториями

In [ ]:
# Задаем необходимые группы
targets = {
    'a. W 25-54':'sex = 2 AND age >= 25 AND age <= 54',
    'b. All 18-64':'age >= 18 AND age <= 64',
    'c. M25-45BC':'sex = 1 AND age >= 25 AND age <= 45 AND incomeGroupRussia IN (2,3)'
}

## Расчет задания

In [ ]:
# Посчитаем задания в цикле
tasks = []
print("Отправляем задания на расчет")

# Для каждой ЦА формируем задание и отправляем на расчет
for target, syntax in targets.items():
    
    # Подставляем значения словаря в параметры
    project_name = target 
    basedemo_filter = syntax
      
    # Формируем задание для API TV Index в формате JSON
    task_json = mtask.build_simple_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                        daytype_filter=daytype_filter, company_filter=company_filter, 
                                        location_filter=location_filter, basedemo_filter=basedemo_filter, 
                                        targetdemo_filter=targetdemo_filter,program_filter=program_filter, 
                                        break_filter=break_filter, ad_filter=ad_filter, 
                                        slices=slices, statistics=statistics, sortings=sortings, options=options)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name    
    tsk['task'] = mtask.send_simple_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '')
    
print(f"\nid: {[i['task']['taskId'] for i in tasks]}") 

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])        
    results.append(df_result)
    print('.', end = '')
df = pd.concat(results)

# Приводим порядок столбцов в соответствие с условиями расчета
df = df[['prj_name']+slices+statistics]

df

## Сохраняем в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('03_several_ta')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)